# 第四节 检索增强生成的原理与实现（Retrieval Augmented Generation，RAG）


检索增强生成（Retrieval Augmented Generation），简称 RAG，已经成为当前最火热的LLM应用方案。它是一个为大模型提供外部知识源的概念，这使它们能够生成准确且符合上下文的答案，同时能够减少模型幻觉。

RAG结合了检索（Retrieval）和生成（Generation）两大核心技术，通过这种独特的混合机制，能够在处理复杂的查询和生成任务时，提供更加准确、丰富的信息。无论是在回答复杂的问题，还是在创作引人入胜的故事，RAG都展现了其不可小觑的能力。

* 数据准备阶段：数据提取 >> 文本分割 >> 向量化（embedding) >> 数据入库

* 应用阶段：用户提问 >> 数据检索（召回） >> 注入Prompt >> LLM生成答案|

本课件主要参考了如下两个链接：

https://datawhalechina.github.io/llm-universe/#/C4/1.%20%E7%9F%A5%E8%AF%86%E5%BA%93%E6%96%87%E6%A1%A3%E5%A4%84%E7%90%86

https://blog.csdn.net/qq_43548590/article/details/135512188

其他参考资料：

https://blog.csdn.net/Julialove102123/article/details/135714213

https://www.zhihu.com/tardis/bd/art/656646499?source_id=1001

https://zhuanlan.zhihu.com/p/640936557

In [1]:
#导入语言模型
import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

import pandas as pd
#导入模版
from langchain.prompts import PromptTemplate

#导入聊天模型
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

#输入三个模型各自的key

os.environ["DASHSCOPE_API_KEY"] = ""

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""

In [2]:
model_ty = Tongyi(temperature=0.1)
model_qf = QianfanLLMEndpoint(temperature=0.1)

## 文档的读入

### pdf文档的读入

In [3]:
from langchain.document_loaders import PyPDFLoader

In [4]:
loader_pdf = PyPDFLoader(r"G:\迅雷下载\基于文本数据的汽车造型需求分析_苏佳幸.pdf")
docs_pdf = loader_pdf.load()

Multiple definitions in dictionary at byte 0x32502b for key /MediaBox


In [5]:
type(docs_pdf)

list

In [6]:
len(docs_pdf)

4

In [35]:
type(docs_pdf[0])

langchain_core.documents.base.Document

In [36]:
docs_pdf[0].metadata

{'source': 'G:\\迅雷下载\\基于文本数据的汽车造型需求分析_苏佳幸.pdf', 'page': 0}

In [37]:
docs_pdf[0].page_content[0:1000]

'150  AUTO TIME\nAUTO AFTERMARKET  | 汽车后市场\n1\u3000引言\n随着国家经济的发展，人民的生活品质\n和水平的提高和购买力的增强，汽车的销量\n飞增。国家统计局数据显示，汽车制造业生产较 2021年同比增长 5.5％\n[1]，虽然汽车行\n业发展迅速，但考虑长远发展仍需推动国内汽车行业深层次发展，本文介绍的文本数据法依托消费者的评价进行分析，促进更高效的产品生产，帮助企业占据市场有利地位。\n数字化用户画像是指借助大数据来提取\n用户的相应特征、需求、偏好等内容并建模分析。\n[2]随着社交媒体如微博、论坛等逐渐\n成熟，能够最直观反映用户体验的各种评价不断更新，传统的人工挖掘方式也变得更加困难、效率低下，并且用户评论中含有大量零碎且多样的情感词汇，难以分析。，本文对国内学者季曹婷等人提出的融合多特征TFIDF文本分析的汽车造型需求提取方法和余本功等人提出的在解决问答社区关键词提取问题的想法进行融合，提出了新的方法\n[3]，\n具体流程见图 1。 \n虽然现如今国内对于该方面的研究较少，\n但不难看出将大数据用户画像与汽车造型相融合能够使汽车行业在未来更加繁荣。\n2\u3000研究方法\n此次研究采用了内容分析法[4]。主要通\n过对特定文本中单词和词组的频率计数进行，将定性的文本数据转化为定量的频数。这种方法真实、客观、全面地反映文本内容的本来意义，具有一定的深度。内容分析法经过选择、分类、统计等三个阶段，以爬取搜集的网络评论文本为分析内容，对数据进行预处理，评论进行分句，删减无关、重复评论，得到筛选清理的评论。再对其进行词语提取，将描述性词语及情感词提取出来，经过统计、排序后，绘制得到高频词汇表，依据高频词汇表解读、判断和挖掘信息中所蕴含的本质内容。苏佳幸\u3000李伽熙\u3000李睿思\u3000李俊豪\u3000赵云芸\n通讯作者\n武汉商学院\u3000湖北省武汉市\u3000 430100\n摘\u2003 要：  近年来互联网的快速发展，汽车业已在网络上建立起了自己的“生态圈”，社交媒体逐渐成熟，传统\n的人工挖掘汽车评论的方式显得效率低下。本文运用网络信息搜集技术对用户评论的文本数据进行搜\n集，并基于此对消费者汽车造型需求展开分析，有效地弥补传统方法的缺陷，获得评价的潜在价值和情绪信息，提炼出

### txt文档的读入

In [7]:
from langchain.document_loaders import TextLoader
loader_txt = TextLoader(r'G:\云岚宗.txt', encoding='utf8')
docs_txt = loader_txt.load()

In [8]:
len(docs_txt)

1

In [41]:
print(docs_txt[0].metadata,docs_txt[0].page_content[0:100])

{'source': 'G:\\云岚宗.txt'} 第四章 云岚宗

大厅中,萧战以及三位长老,正在颇为热切的与那位陌生老者交谈着,不过这位老者似乎有什么难以启齿的事情一般,每每到口的话语,都将会有些无奈的咽了回去,而每当这个时候,一旁的娇贵少女,都是


### 网页文档的读入


In [9]:
from langchain_community.document_loaders import WebBaseLoader
WEB_URL = "https://news.ifeng.com/c/8Y3TlIcTsj0"
loader_html = WebBaseLoader(WEB_URL)
docs_html = loader_html.load()

In [44]:
len(docs_html)

1

In [10]:
print(docs_html[0].metadata,docs_html[0].page_content[0:100])

{'source': 'https://news.ifeng.com/c/8Y3TlIcTsj0', 'title': '足协原副主席王登峰被判17年_凤凰网', 'description': '足协原副主席王登峰被判17年', 'language': 'zh'} 足协原副主席王登峰被判17年_凤凰网



首页资讯视频直播凤凰卫视财经娱乐体育时尚汽车房产科技读书文化历史军事旅游佛教更多国学数码健康家居公益教育酒业美食资讯 > 大陆 > 正文足协原副主席王登峰被


## 文档的分割

Langchain 中文本分割器都根据 chunk_size (块大小)和 chunk_overlap (块与块之间的重叠大小)进行分割。

* chunk_size 指每个块包含的字符或 Token（如单词、句子等）的数量

* chunk_overlap 指两个块之间共享的字符数量，用于保持上下文的连贯性，避免分割丢失上下文信息

Langchain 提供多种文档分割方式，区别在怎么确定块与块之间的边界、块由哪些字符/token组成、以及如何测量块大小

RecursiveCharacterTextSplitter(): 按字符串分割文本，递归地尝试按不同的分隔符进行分割文本。

CharacterTextSplitter(): 按字符来分割文本。

MarkdownHeaderTextSplitter(): 基于指定的标题来分割markdown 文件。

TokenTextSplitter(): 按token来分割文本。

SentenceTransformersTokenTextSplitter(): 按token来分割文本。

Language(): 用于 CPP、Python、Ruby、Markdown 等。

NLTKTextSplitter(): 使用 NLTK（自然语言工具包）按句子分割文本。

SpacyTextSplitter(): 使用 Spacy按句子的切割文本。

In [11]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
# 导入递归字符文本分割器
text_splitter_txt = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
# 导入文本
documents_txt = text_splitter_txt.split_documents(docs_txt)

In [13]:
len(documents_txt)

7

In [14]:
documents_txt[0].page_content

'第四章 云岚宗\n\n大厅中,萧战以及三位长老,正在颇为热切的与那位陌生老者交谈着,不过这位老者似乎有什么难以启齿的事情一般,每每到口的话语,都将会有些无奈的咽了回去,而每当这个时候,一旁的娇贵少女,都是忍不住的横了老者一眼…\n\n    倾耳听了一会，萧炎便是有些无聊的摇了摇头…\n\n    “萧炎哥哥，你知道他们的身份吗？”就在萧炎无聊得想要打瞌睡之时，身旁的熏儿，纤指再次翻开古朴的书页，目不斜视的微笑道。\n\n    “你知道？”好奇的转过头来，萧炎惊诧的问道。\n\n    “看见他们袍服袖口处的云彩银剑了么？”微微一笑，熏儿道。\n\n    “哦？”心头一动，萧炎目光转向三人袖口，果然是发现了一道云彩形状的银剑。\n\n    “他们是云岚宗的人？”萧炎惊讶的低声道。'

In [15]:
#计算字数
sum([len(doc.page_content) for doc in documents_txt])

2285

### classwork 1

1. 实现pdf文档的导入，并对其进行分割，输出分割后的段数与某段的内容

2. 实现在线新闻的导入，并对其进行分割，输出分割后的段数与某段的内容

## 文档词向量化与向量数据库

在机器学习和自然语言处理（NLP）中，Embeddings（嵌入）是一种将类别数据，如单词、句子或者整个文档，转化为实数向量的技术。这些实数向量可以被计算机更好地理解和处理。嵌入背后的主要想法是，相似或相关的对象在嵌入空间中的距离应该很近。

举个例子，我们可以使用词嵌入（word embeddings）来表示文本数据。在词嵌入中，每个单词被转换为一个向量，这个向量捕获了这个单词的语义信息。例如，"king" 和 "queen" 这两个单词在嵌入空间中的位置将会非常接近，因为它们的含义相似。而 "apple" 和 "orange" 也会很接近，因为它们都是水果。而 "king" 和 "apple" 这两个单词在嵌入空间中的距离就会比较远，因为它们的含义不同。

让我们取出我们的切分部分并对它们进行 Embedding 处理。

langchain官网提供了很多向量化的模型：
https://python.langchain.com/docs/integrations/text_embedding

这里我们使用其中的两种

1. baidu_qianfan_endpoint

https://python.langchain.com/docs/integrations/text_embedding/baidu_qianfan_endpoint

2. Hugging Face

https://python.langchain.com/docs/integrations/text_embedding/huggingfacehub


### 百度提供的向量化接口（key只要导入之前大模型的就可以了）


In [16]:
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
import numpy as np

embeddings_qf = QianfanEmbeddingsEndpoint()

In [17]:
query1 = "狗"
query2 = "猫"
query3 = "雨"

# 通过对应的 embedding 类生成 query 的 embedding。
emb1 = embeddings_qf.embed_query(query1)
emb2 = embeddings_qf.embed_query(query2)
emb3 = embeddings_qf.embed_query(query3)

[INFO] [03-19 09:42:51] openapi_requestor.py:316 [t:3856]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-19 09:42:51] oauth.py:207 [t:3856]: trying to refresh access_token for ak `og6mWr***`
[INFO] [03-19 09:42:52] oauth.py:220 [t:3856]: sucessfully refresh access_token
[INFO] [03-19 09:42:52] openapi_requestor.py:316 [t:3856]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-19 09:42:53] openapi_requestor.py:316 [t:3856]: requesting llm api endpoint: /embeddings/embedding-v1


In [18]:
# 将返回结果转成 numpy 的格式，便于后续计算
emb1 = np.array(emb1)
emb2 = np.array(emb2)
emb3 = np.array(emb3)

print(np.dot(emb1, emb2))
print(np.dot(emb3, emb2))
print(np.dot(emb1, emb3))

0.47321960541829167
0.3674960308371415
0.21290156815031291


### 离线的向量化模型

In [19]:
# 第一次运行会下载半个G的模型
from langchain.embeddings import HuggingFaceEmbeddings
embeddings_hf = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base")

In [20]:
query1 = "狗"
query2 = "猫"
query3 = "雨"

# 通过对应的 embedding 类生成 query 的 embedding。
emb1 = embeddings_hf.embed_query(query1)
emb2 = embeddings_hf.embed_query(query2)
emb3 = embeddings_hf.embed_query(query3)

# 将返回结果转成 numpy 的格式，便于后续计算
emb1 = np.array(emb1)
emb2 = np.array(emb2)
emb3 = np.array(emb3)

print(np.dot(emb1, emb2))
print(np.dot(emb3, emb2))
print(np.dot(emb1, emb3))

320.0446771489675
302.9192630617566
300.0346646835118


### 向量数据库（Chroma）


向量数据库是用于高效计算和管理大量向量数据的解决方案。向量数据库是一种专门用于存储和检索向量数据（embedding）的数据库系统。它与传统的基于关系模型的数据库不同，它主要关注的是向量数据的特性和相似性。

在向量数据库中，数据被表示为向量形式，每个向量代表一个数据项。这些向量可以是数字、文本、图像或其他类型的数据。向量数据库使用高效的索引和查询算法来加速向量数据的存储和检索过程。

Langchain 集成了超过 30 个不同的向量存储库。我们选择 Chroma 是因为它轻量级且数据存储在内存中，这使得它非常容易启动和开始使用。

langchain支持列表

https://python.langchain.com/docs/integrations/vectorstores



#### 数据库的创建与载入

In [ ]:
#! pip install chromadb -i https://mirrors.aliyun.com/pypi/simple

In [12]:
#help(Chroma.from_documents)

In [21]:
from langchain.vectorstores import Chroma
# 导入千帆向量模型
embeddings_qf = QianfanEmbeddingsEndpoint()

#persist_directory允许我们将目录保存到磁盘上
#注意from_documents每次运行都是把数据添加进去
vectordb = Chroma.from_documents(documents=documents_txt, embedding=embeddings_qf, persist_directory="G:\\" )

[INFO] [03-19 09:44:12] openapi_requestor.py:316 [t:3856]: requesting llm api endpoint: /embeddings/embedding-v1


In [18]:
vectordb.persist()

In [22]:
vectordb_load = Chroma(
    persist_directory="G:\\",
    embedding_function=embeddings_qf
)

In [23]:
vectordb_load._collection.count()

14

#### 通过向量数据库检索

##### 相似度检索

In [24]:
vectordb_load.similarity_search("云岚宗")

[INFO] [03-19 09:44:34] openapi_requestor.py:316 [t:3856]: requesting llm api endpoint: /embeddings/embedding-v1


[Document(page_content='第四章 云岚宗\n\n大厅中,萧战以及三位长老,正在颇为热切的与那位陌生老者交谈着,不过这位老者似乎有什么难以启齿的事情一般,每每到口的话语,都将会有些无奈的咽了回去,而每当这个时候,一旁的娇贵少女,都是忍不住的横了老者一眼…\n\n    倾耳听了一会，萧炎便是有些无聊的摇了摇头…\n\n    “萧炎哥哥，你知道他们的身份吗？”就在萧炎无聊得想要打瞌睡之时，身旁的熏儿，纤指再次翻开古朴的书页，目不斜视的微笑道。\n\n    “你知道？”好奇的转过头来，萧炎惊诧的问道。\n\n    “看见他们袍服袖口处的云彩银剑了么？”微微一笑，熏儿道。\n\n    “哦？”心头一动，萧炎目光转向三人袖口，果然是发现了一道云彩形状的银剑。\n\n    “他们是云岚宗的人？”萧炎惊讶的低声道。', metadata={'source': 'G:\\云岚宗.txt'}),
 Document(page_content='第四章 云岚宗\n\n大厅中,萧战以及三位长老,正在颇为热切的与那位陌生老者交谈着,不过这位老者似乎有什么难以启齿的事情一般,每每到口的话语,都将会有些无奈的咽了回去,而每当这个时候,一旁的娇贵少女,都是忍不住的横了老者一眼…\n\n    倾耳听了一会，萧炎便是有些无聊的摇了摇头…\n\n    “萧炎哥哥，你知道他们的身份吗？”就在萧炎无聊得想要打瞌睡之时，身旁的熏儿，纤指再次翻开古朴的书页，目不斜视的微笑道。\n\n    “你知道？”好奇的转过头来，萧炎惊诧的问道。\n\n    “看见他们袍服袖口处的云彩银剑了么？”微微一笑，熏儿道。\n\n    “哦？”心头一动，萧炎目光转向三人袖口，果然是发现了一道云彩形状的银剑。\n\n    “他们是云岚宗的人？”萧炎惊讶的低声道。', metadata={'source': 'G:\\云岚宗.txt'}),
 Document(page_content='虽然并没有外出历练，不过萧炎在一些书籍中却看过有关这剑派的资料，萧家所在的城市名为乌坦城，乌坦城隶属于加玛帝国，虽然此城因为背靠魔兽山脉的地利，而跻身进入帝国的大城市之列，不过也仅仅只是居于末座。\n\n    萧炎的家族，在乌坦城颇有份量，不过却也并不是唯一，城市中，还有另外两大家族实力与萧家相差无几，三

#####  MMR 检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (MMR, Maximum marginal relevance) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [22]:
vectordb_load.max_marginal_relevance_search("云岚宗")


[INFO] [03-18 21:57:37] openapi_requestor.py:316 [t:11216]: requesting llm api endpoint: /embeddings/embedding-v1
Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


[Document(page_content='第四章 云岚宗\n\n大厅中,萧战以及三位长老,正在颇为热切的与那位陌生老者交谈着,不过这位老者似乎有什么难以启齿的事情一般,每每到口的话语,都将会有些无奈的咽了回去,而每当这个时候,一旁的娇贵少女,都是忍不住的横了老者一眼…\n\n    倾耳听了一会，萧炎便是有些无聊的摇了摇头…\n\n    “萧炎哥哥，你知道他们的身份吗？”就在萧炎无聊得想要打瞌睡之时，身旁的熏儿，纤指再次翻开古朴的书页，目不斜视的微笑道。\n\n    “你知道？”好奇的转过头来，萧炎惊诧的问道。\n\n    “看见他们袍服袖口处的云彩银剑了么？”微微一笑，熏儿道。\n\n    “哦？”心头一动，萧炎目光转向三人袖口，果然是发现了一道云彩形状的银剑。\n\n    “他们是云岚宗的人？”萧炎惊讶的低声道。', metadata={'source': 'G:\\云岚宗.txt'}),
 Document(page_content='虽然并没有外出历练，不过萧炎在一些书籍中却看过有关这剑派的资料，萧家所在的城市名为乌坦城，乌坦城隶属于加玛帝国，虽然此城因为背靠魔兽山脉的地利，而跻身进入帝国的大城市之列，不过也仅仅只是居于末座。\n\n    萧炎的家族，在乌坦城颇有份量，不过却也并不是唯一，城市中，还有另外两大家族实力与萧家相差无几，三方彼此明争暗斗了几十年，也未曾分出胜负…\n\n    如果说萧家是乌坦城的一霸，那么萧炎口中所说的云岚宗，或许便应该说是整个加玛帝国的一霸！这之间的差距，犹如鸿沟，也难怪连平日严肃的父亲，在言语上很是敬畏。\n\n    “他们来我们家族做什么？”萧炎有些疑惑的低声询问道。\n\n    移动的纤细指尖微微一顿，熏儿沉默了一会，方才道：“或许和萧炎哥哥有关…”\n\n    “我？我可没和他们有过什么交集啊？”闻言，萧炎一怔，摇头否认。', metadata={'source': 'G:\\云岚宗.txt'}),
 Document(page_content='望着熏儿的躲避态势，萧炎只得无奈的撇了撇嘴，熏儿虽然也姓萧，不过与他却没有半点血缘关系，而且熏儿的父母，萧炎也从未见过，每当他询问自己的父亲时，满脸笑容的父亲便会立刻闭口不语，显然对熏儿的父母很是忌讳，甚至…惧怕！\n\n    在萧炎心中，熏儿的身份

### classwork 2

1. 基于pdf文档的导入与分割后，向量化对应文档并将其存入向量数据库并持久化（建议不同文档不同目录），然后尝试通过相似性进行检索

2. 对在线新闻实现上题一样的操作，并尝试通过相似性进行检索

## 构造检索式问答链


* 四种文档链：Stuff；Refine ；Map reduce；Map re-rank

https://python.langchain.com.cn/docs/modules/chains/document/


* Stuff documents

https://python.langchain.com.cn/docs/modules/chains/document/stuff

In [25]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [35]:
# 创建提示词模板
prompt = ChatPromptTemplate.from_template("""使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话，你的回答中不可以带有星号。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
<context>
{context}
</context>

Question: {input}""")
#创建检索链
document_chain = create_stuff_documents_chain(model_qf, prompt)

retriever = vectordb_load.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [28]:
response = retrieval_chain.invoke({
    "input": "萧炎是怎样的强者?"
})
print(response["answer"])

[INFO] [03-19 09:44:58] openapi_requestor.py:316 [t:22552]: requesting llm api endpoint: /embeddings/embedding-v1


[INFO] [03-19 09:44:58] openapi_requestor.py:316 [t:15196]: requesting llm api endpoint: /chat/eb-instant


根据语料信息，萧炎是一位有潜力的强者，但目前并没有外出历练。他在乌坦城和加玛帝国有一定的地位，但并不是唯一的一家。他口中所说的云岚宗在加玛帝国是一霸，这表明他有一定的实力和背景。此外，他对于纳兰嫣然的解除婚约请求感到愤怒，这表明他有一定的自尊心和保护自己尊严的决心。因此，可以说萧炎是一位有潜力、有背景、有自尊心的强者。


#### 构建一个检索式问答RAG的全过程

In [29]:
# 构建一个检索式问答RAG的全过程

model_qf = QianfanLLMEndpoint(temperature=0.1)
loader_txt = TextLoader(r'G:\云岚宗.txt', encoding='utf8')
docs_txt = loader_txt.load()
text_splitter_txt = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
documents_txt = text_splitter_txt.split_documents(docs_txt)
embeddings_qf = QianfanEmbeddingsEndpoint()
vectordb = Chroma.from_documents(documents=documents_txt, embedding=embeddings_qf, persist_directory="G:\\" )

prompt = ChatPromptTemplate.from_template("""使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话，你的回答中不可以带有星号。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
<context>
{context}
</context>

Question: {input}""")
#创建检索链
document_chain = create_stuff_documents_chain(model_qf, prompt)

retriever = vectordb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

[INFO] [03-19 09:45:05] openapi_requestor.py:316 [t:3856]: requesting llm api endpoint: /embeddings/embedding-v1


#### 构建一个检索式文档对话模型

In [36]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(
    llm=model_qf, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm=model_qf, retriever=retriever, memory=memory)

In [38]:
res = qa.invoke(
    {"question": "萧炎是谁"}
)
print(res["answer"])

[INFO] [03-19 09:46:37] openapi_requestor.py:316 [t:3856]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-19 09:46:37] openapi_requestor.py:316 [t:3856]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-19 09:46:37] openapi_requestor.py:316 [t:3856]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-19 09:46:40] openapi_requestor.py:316 [t:3856]: requesting llm api endpoint: /chat/eb-instant


根据原文信息得出，萧炎是故事中的一个人物，是主人公萧炎的昵称。文中提到，虽然萧炎并没有外出历练，但他通过一些书籍了解到了有关剑派的资料，并且他的家族位于乌坦城，这是一个隶属于加玛帝国的大城市。文中还提到，萧炎的家族在乌坦城颇有份量，但并不是唯一的家族，还有另外两个实力与萧家相差无几的家族。文中还提到，萧炎口中所说的云岚宗可能是整个加玛帝国的一霸，这之间的差距犹如鸿沟。此外，文中还提到了熏儿和纳兰嫣然等人物。因此可以得出结论，萧炎是故事中的主人公。


In [28]:
res = qa.invoke(
    {"question": "他结婚了吗？"}
)
print(res["answer"])

[INFO] [03-18 23:09:44] openapi_requestor.py:316 [t:11216]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-18 23:09:44] openapi_requestor.py:316 [t:11216]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-18 23:09:45] openapi_requestor.py:316 [t:11216]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-18 23:09:47] openapi_requestor.py:316 [t:11216]: requesting llm api endpoint: /chat/eb-instant


根据原文信息得出，萧炎还没有结婚。文中提到，纳兰嫣然是纳兰桀的孙女，也是萧炎的未婚妻，但文中提到纳兰桀这老头不仅性子桀骜，而且为人又极其在乎承喏，当年的婚事，是他亲口应下来的，所以就算萧炎最近几年名声极差，他也未曾派人过来悔婚。因此可以得出结论，萧炎目前还没有结婚。


### classwork 3

1. 基于上面新闻页面实现检索式的文档问答

2. 基于上面新闻页面实现基于检索式的聊天问答

## 